In [20]:
import requests
import pandas as pd
import folium
from branca.colormap import LinearColormap

# ───────────────────────────────────────────────────────────────
# PARAMETERS
FETCH_LIMIT      = 50000    # rows to fetch
SAMPLE_LOCATIONS = 200      # distinct spots to plot
BASE_RADIUS      = 2        # minimum circle radius
SCALE            = 1.2      # multiplier for √count → radius
# ───────────────────────────────────────────────────────────────

# 1. Fetch centroid data
url    = "https://data.cityofchicago.org/resource/ajtu-isnz.json"
params = {"$limit": FETCH_LIMIT}
resp   = requests.get(url, params=params)
resp.raise_for_status()
df     = pd.DataFrame(resp.json())

# 2. Coerce coords to numeric, drop invalid
df['pickup_lat']  = pd.to_numeric(df['pickup_centroid_latitude'],  errors='coerce')
df['pickup_lng']  = pd.to_numeric(df['pickup_centroid_longitude'], errors='coerce')
df['dropoff_lat'] = pd.to_numeric(df['dropoff_centroid_latitude'],  errors='coerce')
df['dropoff_lng'] = pd.to_numeric(df['dropoff_centroid_longitude'], errors='coerce')
df = df.dropna(subset=['pickup_lat','pickup_lng','dropoff_lat','dropoff_lng'])

# 3. Aggregate trip counts
pickup_counts = df.groupby(['pickup_lat','pickup_lng']).size().reset_index(name='count')
dropoff_counts = df.groupby(['dropoff_lat','dropoff_lng']).size().reset_index(name='count')

# 4. Sample for performance
pickup_counts  = pickup_counts.sample(n=min(SAMPLE_LOCATIONS, len(pickup_counts)), random_state=42)
dropoff_counts = dropoff_counts.sample(n=min(SAMPLE_LOCATIONS, len(dropoff_counts)), random_state=42)

# 5. Sort so smaller circles draw first, larger on top
pickup_counts  = pickup_counts.sort_values('count')
dropoff_counts = dropoff_counts.sort_values('count')

# ───────────── PICKUP MAP ──────────────────────────────────────
p_min, p_max = pickup_counts['count'].min(), pickup_counts['count'].max()
pickup_map = folium.Map(location=[pickup_counts['pickup_lat'].mean(),
                                  pickup_counts['pickup_lng'].mean()],
                        zoom_start=11)

# use yellow→orange→red for stronger midtones
pickup_cmap = LinearColormap(['yellow','orange','red'],
                             vmin=p_min, vmax=p_max,
                             caption='Pickups')
pickup_cmap.add_to(pickup_map)

for _, row in pickup_counts.iterrows():
    folium.CircleMarker(
        location=[row['pickup_lat'], row['pickup_lng']],
        radius= BASE_RADIUS + (row['count']**0.3) * SCALE,
        color=pickup_cmap(row['count']),
        fill=True,
        fill_color=pickup_cmap(row['count']),
        fill_opacity=0.7,
        popup=f"Pickups: {row['count']}"
    ).add_to(pickup_map)

#pickup_map.save("chicago_taxi_pickups_scaled.html")


# ──────────── DROPOFF MAP ───────────────────────────────────────
d_min, d_max = dropoff_counts['count'].min(), dropoff_counts['count'].max()
dropoff_map = folium.Map(location=[dropoff_counts['dropoff_lat'].mean(),
                                   dropoff_counts['dropoff_lng'].mean()],
                         zoom_start=11)

dropoff_cmap = LinearColormap(['yellow','orange','red'],
                              vmin=d_min, vmax=d_max,
                              caption='Drop-offs')
dropoff_cmap.add_to(dropoff_map)

for _, row in dropoff_counts.iterrows():
    folium.CircleMarker(
        location=[row['dropoff_lat'], row['dropoff_lng']],
        radius= BASE_RADIUS + (row['count']**0.3) * SCALE,
        color=dropoff_cmap(row['count']),
        fill=True,
        fill_color=dropoff_cmap(row['count']),
        fill_opacity=0.7,
        popup=f"Drop-offs: {row['count']}"
    ).add_to(dropoff_map)

#dropoff_map.save("chicago_taxi_dropoffs_scaled.html")

pickup_map



In [21]:
dropoff_map